In [1]:
import sys
sys.path.append('../input/multilabelstraifier/')
from ml_stratifiers import MultilabelStratifiedKFold

import pandas as pd
import numpy as np
from sklearn.metrics import log_loss
import matplotlib.pyplot as plt
from sklearn.metrics import log_loss
DIR = "../input/moa-ensemble-check/"
check_dae = pd.read_csv(DIR+"dae_oof3_chris.csv").drop("sig_id", axis=1)
check_mlp = pd.read_csv(DIR+"mlp_oof2.csv")
check_tabnet = pd.read_csv(DIR+"tab_oof2.csv")

y = pd.read_csv('../input/lish-moa/train_targets_scored.csv')
targets = pd.read_csv('../input/lish-moa/train_targets_scored.csv').drop("sig_id", axis=1)
train = pd.read_csv('../input/lish-moa/train_features.csv')
drug = pd.read_csv('../input/lish-moa/train_drug.csv')

#mlp = pd.read_csv(DIR+"mlp1840.csv")
#check_tabnet = pd.read_csv(DIR+"tab_oof2.csv")
#tab = pd.read_csv(DIR+"sub_tab1859.csv")
#xgb = pd.read_csv(DIR+"xgb1951.csv")
#check_xgb = pd.read_csv(DIR+"xgb_oof.csv")

In [2]:
cons_train_index = train[train.cp_type!="ctl_vehicle"].index

In [3]:
y = y.loc[y.index.isin(cons_train_index),:]

In [4]:
corr_df = y.corr().abs()
corr_df

,5-alpha_reductase_inhibitor,11-beta-hsd1_inhibitor,acat_inhibitor,acetylcholine_receptor_agonist,acetylcholine_receptor_antagonist,acetylcholinesterase_inhibitor,adenosine_receptor_agonist,adenosine_receptor_antagonist,adenylyl_cyclase_activator,adrenergic_receptor_agonist,...,tropomyosin_receptor_kinase_inhibitor,trpv_agonist,trpv_antagonist,tubulin_inhibitor,tyrosine_kinase_inhibitor,ubiquitin_specific_protease_inhibitor,vegfr_inhibitor,vitamin_b,vitamin_d_receptor_agonist,wnt_inhibitor
5-alpha_reductase_inhibitor,1.000000,0.000798,0.000921,0.002602,0.003283,0.001608,0.001383,0.001845,0.000651,0.003107,...,0.000460,0.000940,0.001303,0.003365,0.001608,0.000460,0.002460,0.000959,0.001175,0.001030
11-beta-hsd1_inhibitor,0.000798,1.000000,0.000948,0.002677,0.003378,0.164210,0.001423,0.001899,0.000670,0.003197,...,0.000474,0.000967,0.001341,0.003463,0.001655,0.000474,0.002531,0.000987,0.001209,0.001060
acat_inhibitor,0.000921,0.000948,1.000000,0.003092,0.003901,0.001911,0.001643,0.002193,0.000774,0.003692,...,0.000547,0.001117,0.001549,0.003999,0.001911,0.000547,0.002923,0.001139,0.001396,0.001224
acetylcholine_receptor_agonist,0.002602,0.002677,0.003092,1.000000,0.011019,0.005398,0.004641,0.006194,0.002186,0.010429,...,0.001545,0.003156,0.004375,0.011294,0.005398,0.001545,0.008256,0.003218,0.003943,0.003457
acetylcholine_receptor_antagonist,0.003283,0.003378,0.003901,0.011019,1.000000,0.006812,0.005856,0.007816,0.002758,0.013160,...,0.001950,0.003982,0.005521,0.014252,0.006812,0.001950,0.010418,0.004061,0.004975,0.004363
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ubiquitin_specific_protease_inhibitor,0.000460,0.000474,0.000547,0.001545,0.001950,0.000955,0.000821,0.001096,0.000387,0.001845,...,0.000273,0.000558,0.000774,0.001999,0.000955,1.000000,0.001461,0.000569,0.000698,0.000612
vegfr_inhibitor,0.002460,0.002531,0.002923,0.008256,0.010418,0.005104,0.004388,0.005856,0.002066,0.009860,...,0.001461,0.002984,0.004136,0.010679,0.005104,0.001461,1.000000,0.003043,0.003728,0.003269
vitamin_b,0.000959,0.000987,0.001139,0.003218,0.004061,0.001989,0.001710,0.002283,0.000805,0.003843,...,0.000569,0.001163,0.001612,0.004162,0.001989,0.000569,0.003043,1.000000,0.001453,0.001274
vitamin_d_receptor_agonist,0.001175,0.001209,0.001396,0.003943,0.004975,0.002437,0.002095,0.002796,0.000987,0.004709,...,0.000698,0.001425,0.001975,0.005099,0.002437,0.000698,0.003728,0.001453,1.000000,0.001561


In [5]:
p_min = 0.001
p_max = 1 - p_min

check_tabnet2 = check_tabnet.copy()
check_tabnet2.loc[cons_train_index,:] = np.clip(check_tabnet.loc[cons_train_index,:], p_min, p_max)

# output correlation check

In [6]:
print("mlp dae")
result = []
for i in range(206):
    result.append(np.abs(np.corrcoef(check_mlp.iloc[:,i], check_dae.iloc[:,i])[0,1]))
print(np.max(result), np.min(result), np.mean(result), np.std(result))
print("mlp tab")
result = []
for i in range(206):
    result.append(np.corrcoef(check_mlp.iloc[:,i], check_tabnet.iloc[:,i])[0,1])
print(np.max(result), np.min(result), np.mean(result), np.std(result))

print("tab dae")
result = []
for i in range(206):
    result.append(np.corrcoef(check_tabnet.iloc[:,i], check_dae.iloc[:,i])[0,1])
print(np.max(result), np.min(result), np.mean(result), np.std(result))

mlp dae
0.9857633353621473 0.00024035740939252174 0.20199225165891402 0.20664417386215728
mlp tab
0.9959539601157769 0.0839028380494226 0.5295034711801632 0.17906306979966444
tab dae
0.6720363795485437 -0.26047384953345076 0.17593079826549524 0.19735055990743638


# ensemble ratio

In [7]:
print('OOF log loss: ', log_loss(np.ravel(targets), np.ravel(check_mlp)))
print('OOF log loss: ', log_loss(np.ravel(targets), np.ravel(check_tabnet)))

OOF log loss:  0.01567181329219744
OOF log loss:  0.015617277282027095
